In [6]:
import pandas as pd
import keras
import numpy as np
import matplotlib.pyplot as plt

In [22]:
# Importing the dataset
dataset_clean = pd.read_csv('../datasets_clean/incidents_clean.csv', low_memory=True, index_col=0)
dataset_original=pd.read_csv('../datasets_original/incidents.csv', low_memory=False)


In [23]:
dataset_clean['avg_age_participants'] = dataset_original['avg_age_participants']


In [24]:
def convert_to_float(x):
    """
    This function converts a string to float if possible, and to NaN otherwise.
    """
    try:
        return float(x)

    except ValueError:
        return np.nan

dataset_clean['avg_age_participants'] = dataset_clean['avg_age_participants'].apply(convert_to_float)

In [25]:
#Clean ages by deleting negative values and outliers
dataset_clean['avg_age_participants'] = dataset_clean['avg_age_participants'].apply(lambda x: np.nan if (x<0 or x>100)  else x)

In [26]:
# split the dataset into known and unknown ages
dataset_known_ages = dataset_clean[dataset_clean['avg_age_participants'].notnull()]
dataset_unknown_ages = dataset_clean[dataset_clean['avg_age_participants'].isnull()]


In [27]:

# drop columns with missing values
dataset_known_ages = dataset_known_ages.dropna(axis=1, how='any')


In [28]:
dataset_known_ages.head()

,state,city_or_county,latitude,longitude,avg_age_participants,n_participants_child,n_participants_teen,n_participants_adult,n_males,n_females,n_killed,n_injured,n_arrested,n_unharmed,n_participants,incident_characteristics1
0,Indiana,Indianapolis,39.8322,-86.2492,19.0,0.0,0.0,1.0,1.0,0.0,0,1,0.0,0.0,1.0,Shot - Wounded/Injured
1,Pennsylvania,Kane,41.6645,-78.7856,62.0,0.0,0.0,1.0,1.0,0.0,1,0,0.0,0.0,1.0,"Shot - Dead (murder, accidental, suicide)"
5,North Carolina,Wayne County,35.1847,-77.9527,65.0,0.0,0.0,1.0,1.0,0.0,0,1,0.0,0.0,1.0,Shot - Wounded/Injured
6,Louisiana,Zachary,30.6069,-91.2270,26.0,0.0,0.0,5.0,5.0,0.0,2,1,2.0,0.0,5.0,Shot - Wounded/Injured
8,Virginia,Stafford,38.3008,-77.4469,65.0,0.0,0.0,1.0,1.0,0.0,1,0,0.0,0.0,1.0,"Shot - Dead (murder, accidental, suicide)"


In [37]:
# X= other columns
# y= avg_age_participants, located at column 4
columns_X=dataset_known_ages.columns.drop('avg_age_participants')
X= dataset_known_ages[columns_X]
y= dataset_known_ages['avg_age_participants']

In [38]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [39]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test= sc.transform(X_test)

ValueError: could not convert string to float: 'Illinois'

In [36]:
X_train

NameError: name 'X_train' is not defined